In [19]:
import numpy as np
np.__version__
import math

In [28]:
class TicTacToe:
    def __init__(self):
        self.row_count = 10
        self.column_count = 10
        self.action_size = self.row_count * self.column_count
        
    def get_initial_state(self):
        return np.zeros((self.row_count, self.column_count))
    
    def get_next_state(self, state, action, player):
        row = action // self.column_count
        column = action % self.column_count
        state[row, column] = player
        return state
    
    def get_valid_moves(self, state):
        return (state.reshape(-1) == 0).astype(np.uint8)
    
    def check_win(self, state, action):
        # case: root node always has child
        if action == None:
            return False
        
        row = action // self.column_count
        column = action % self.column_count
        player = state[row, column]
        
        return (
            np.sum(state[row, :]) == player * self.column_count
            or np.sum(state[:, column]) == player * self.row_count
            or np.sum(np.diag(state)) == player * self.row_count
            or np.sum(np.diag(np.flip(state, axis=0))) == player * self.row_count
        )
    
    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        return 0, False
    
    def get_opponent(self, player):
        return -player
    
    def get_opponent_value(self, value):
        return -value
    
    def change_perspective(self, state, player):
        return state * player

In [24]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None):
        # root node doesn't have parent and action taken
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        
        # store the children inside our node
        self.children = []
        # which ways we could further expand or not
        self.expandable_moves = game.get_valid_moves(state)
        
        self.visit_count = 0
        self.value_sum = 0
        
    def is_fully_expanded(self):
        # còn đi được bước nào nữa k
        # có children k 
        return np.sum(self.expandable_moves) == 0 and len(self.children) > 0
    
    # loop all over all of our children as a node
    # for each child, calculate the UCB score
    # pick the child having higest UCB score 
    def select(self):
        best_child = None
        best_ucb = -np.inf
        
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
       
        return best_child
    
    # calculate the UCB score 
    def get_ucb(self, child):
        # q_value: likelihood of winning for a given node
        # AI(player -1) get the negative value for value_sum but q_value: [0,1]
        q_value = 1 - ((child.value_sum / child.visit_count) + 1) / 2
        # constant C: whether we want to focus on exploration or exploitation
        return q_value + self.args['C'] * math.sqrt(math.log(self.visit_count) / child.visit_count)
    
    
    def expand(self):
        # np.where: get the indices for all legal moves
        action = np.random.choice(np.where(self.expandable_moves == 1)[0])
        self.expandable_moves[action] = 0
        
        child_state = self.state.copy()
        # not change player 
        # but change the state of our children (flip the value)
        child_state = self.game.get_next_state(child_state, action, 1)
        child_state = self.game.change_perspective(child_state, player=-1)
        
        child = Node(self.game, self.args, child_state, self, action)
        self.children.append(child)
        return child
    
    def simulate(self):
        value, is_terminal = self.game.get_value_and_terminated(self.state, self.action_taken)
        value = self.game.get_opponent_value(value)
        
        if is_terminal:
            return value
        
        rollout_state = self.state.copy()
        rollout_player = 1
        while True:
            valid_moves = self.game.get_valid_moves(rollout_state)
            action = np.random.choice(np.where(valid_moves == 1)[0])
            rollout_state = self.game.get_next_state(rollout_state, action, rollout_player)
            value, is_terminal = self.game.get_value_and_terminated(rollout_state, action)
            if is_terminal:
                if rollout_player == -1:
                    value = self.game.get_opponent_value(value)
                return value    
            
            rollout_player = self.game.get_opponent(rollout_player)
    
    # recursive
    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        
        # parent: different player 
        value = self.game.get_opponent_value(value)
        # always true except the root node
        if self.parent is not None:
            self.parent.backpropagate(value)  


class MCTS:
    def __init__(self, game, args):
        self.game = game
        self.args = args
    
    # get MCTS-distributions
    def search(self, state):
        # define a root node
        root = Node(self.game, self.args, state)
        
        # SELECTION: 
        # keep selecting downwards the tree as long as our nodes aare fully expanded
        # do all of our search iterations
        for search in range(self.args['num_searches']):
            # selection
            node = root
            
            while node.is_fully_expanded():
                node = node.select()
            
            # check node is terminal?
            value, is_terminal = self.game.get_value_and_terminated(node.state, node.action_taken)
            value = self.game.get_opponent_value(value)
            
            # EXPANSION
            if not is_terminal:
                node = node.expand()
                # SIMULATION: rollouts
                # random actions until reach terminal node
                # look at the value (who won?) to backpropagation
                value = node.simulate()
            
            # BACKPROPAGATE
            # add a value to a value_sum
            # +1 visit
            # keep all the way to root
            node.backpropagate(value)    
            
        # return visit_counts    
        action_probs = np.zeros(self.game.action_size)
        # loop of all children
        for child in root.children:
            action_probs[child.action_taken] = child.visit_count
        # the probability
        action_probs /= np.sum(action_probs)
        return action_probs

In [ ]:
tictactoe = TicTacToe()
player = 1

args = {
    'C': 1.41,
    'num_searches': 1000
}

mcts = MCTS(tictactoe, args)

state = tictactoe.get_initial_state()


while True:
    print(state)
    
    if player == 1:
        valid_moves = tictactoe.get_valid_moves(state)
        print("valid_moves", [i for i in range(tictactoe.action_size) if valid_moves[i] == 1])
        action = int(input(f"{player}:"))

        if valid_moves[action] == 0:
            print("action not valid")
            continue
            
    else:
        neutral_state = tictactoe.change_perspective(state, player)
        mcts_probs = mcts.search(neutral_state)
        # choose the node visited most # times
        action = np.argmax(mcts_probs)
        
    state = tictactoe.get_next_state(state, action, player)
    
    value, is_terminal = tictactoe.get_value_and_terminated(state, action)
    
    if is_terminal:
        print(state)
        if value == 1:
            print(player, "won")
        else:
            print("draw")
        break
        
    player = tictactoe.get_opponent(player)